## Example of how to import our google sheets to python

In [1]:
#ipython magic
%reset -f
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import sys
sys.path.append('..')
from utils.gsheets_importer import gsheet2df, correct_behaviour_df
import numpy as np
import os
from paq2py import *
import matplotlib.pyplot as plt
from utils.opto_stim_import2 import OptoStim1p, OptoStim2p 

from paq2py import *

import matplotlib.pyplot as plt


In [5]:
#### spreadsheet settings

#from the URL
sheet_ID = '1GG5Y0yCEw_h5dMfHBnBRTh5NXgF6NqC_VVGfSpCHroc'

#the name of the file path column
t_series_header = 't-series name'
paq_header = '.paq file'
naparm_header = 'Naparm'

#column telling whether to analyse or not
analyse_bool_header = 'Analyse'
suite2p_bool_header = 'Suite2p me?'

In [6]:
#the sheet is named after the mouse in my spreadsheet
mouse_name = 'RL019'
#have to give it rows and columns, so just given a huge range here
SHEET_NAME = mouse_name + '!A1:Z69'

df = gsheet2df(sheet_ID, HEADER_ROW=2, SHEET_NAME=SHEET_NAME)
#comment this out if not using behaviour spreadsheet
df = correct_behaviour_df(df)
df

,Date,Trained 1P,Trained 2P,Total rewards,Run Number,pycontrol txt,pycontrol task,Analyse,Comments,Start Autoreward,...,Spirals,Repeats,inter group interval (ms),Power / cell (mW),blimp folder,Naparm,.paq file,t-series name,Number of frames,Suite2p me?
0,2019-04-04,TRUE,FALSE,79,1,RL019-2019-04-04-155006,opto_stim,TRUE,Got out of autoreward phase but didn't get clo...,TRUE,...,,,,,,,,,,
1,2019-04-05,TRUE,FALSE,84,2,RL019-2019-04-05-154423,opto_stim,TRUE,Smashed through to the end in an hour. Maybe s...,FALSE,...,,,,,,,,,,
2,2019-04-06,TRUE,FALSE,99,3,RL019-2019-04-06-220134,opto_stim_psychometric,TRUE,Setup looked to be working fine but gui crashed,FALSE,...,,,,,,,,,,
3,2019-04-06,TRUE,FALSE,99,4,RL019-2019-04-06-221238,opto_stim_psychometric,TRUE,Experiment appeared to work well mouse scored ...,FALSE,...,,,,,,,,,,
4,2019-04-07,TRUE,FALSE,84,5,RL019-2019-04-07-170016,opto_stim_psychometric,TRUE,Experiment appeared to work well mouse scored ...,FALSE,...,,,,,,,,,,
5,2019-04-14,FALSE,TRUE,74,6,RL019-2019-04-15-010151,opto_stim_scope,TRUE,Mouse looks like he got it dprime = 0.65ish. S...,NA,...,10x25ms,1,20,10,2019-04-15-010111,2019-04-14_RL019_NAPARM_001,2019-04-14_RL019_blimp_002.paq,"[2019-04-14_RL0019_blimp_t-002.bin, 2019-04-14...","[35849, 35849, 35849]",FALSE
6,2019-04-15,FALSE,TRUE,Topped up,7,RL019-2019-04-15-203322,opto_stim_scope,FALSE,Shortened lick window to 1s. Mouse was doing a...,NA,...,10x25ms,1,20,10,2019-04-15-203244,2019-04-15_RL019_NAPARM_001,2019-04-15_RL019_001.paq,2019-04-15_RL019_t-002.bin,35849,TRUE
7,2019-04-17,FALSE,TRUE,65,8,RL019-2019-04-17-231415,opto_stim_scope,TRUE,Mouse seemed to have forgotten task at start b...,NA,...,1x15ms,5,10,10,2019-04-17-231140,2019-04-17_RL019_NAPARM_001,2019-04-17_Rl019_001.paq,"[2019-04-17_RL019_t-002.bin, 2019-04-17_RL019_...","[35849, 35849, 18084]",FALSE
8,2019-04-17,FALSE,TRUE,65,9,RL019-2019-04-18-002855,opto_stim_scope,TRUE,Ran for a short time with very good performanc...,NA,...,1x15ms,5,10,10,2019-04-18-002722,2019-04-17_RL019_NAPARM_001,2019-04-17_Rl019_002.paq,2019-04-17_RL019_t-005.bin,15668,FALSE
9,2019-04-19,FALSE,TRUE,74,10,RL019-2019-04-19-190618,opto_stim_scope,TRUE,Mouse did great d-prime = 1.5 even though towa...,NA,...,1x10ms,10,5,10,2019-04-19-190434,2019-04-19_RL019_NAPARM_001,2019-04-19_RL019_001.paq,None,None,FALSE


In [7]:
def split_df(self, col_id):
    '''slice whole dataframe by boolean value of col_id'''
    idx = self.df.index[self.df[col_id]=='TRUE']
    return self.df.loc[idx, :]
    
def df_bool(df, col_id):
    '''returns the rows of a pandas dataframe where col_id is TRUE'''
    return df.index[df[col_id]=='TRUE']

def df_col(df, col, idx='all'):   
    
    '''returns the values in col in rows specified in idx'''
    
    if idx=='all': 
        idx = range(len(df))
        
    return [df.loc[idx,col] for idx in idx]

def intersect(A, B):
    '''returns a list of the common items between two lists'''
    return set(A) - (set(A) - set(B))


In [8]:
idx_analyse = df_bool(df, analyse_bool_header)
idx_2p = df_bool(df, 'Trained 2P')
idx_1p = df_bool(df, 'Trained 1P')

rows_2p = intersect(idx_analyse, idx_2p)
rows_1p = intersect(idx_analyse, idx_1p)

tseries_paths = df_col(df, t_series_header, rows_2p)
paq_paths = df_col(df, paq_header, rows_2p)
naparm_paths = df_col(df, naparm_header, rows_2p)

naparm_paths

['2019-04-17_RL019_NAPARM_001',
 '2019-04-19_RL019_NAPARM_001',
 '2019-04-14_RL019_NAPARM_001',
 '2019-04-17_RL019_NAPARM_001']

In [ ]:
def path_conversion(path_list, packerstation_path = '/home/jamesrowland/Documents/packerstation'):
    
    '''converts local paths on 2p imaging computer to packerstation paths
       only works with data arrangement as of 2019-03-27 will likely break in the future
    '''
    
    p = path_list[0]
    name = p.split('\\')[2]
    date = p.split('\\')[3]
    
    local_path = os.path.join(packerstation_path, name, 'Data', date)
    
    converted_paths = [os.path.join(local_path, *p.split('\\')[4:]) for p in path_list]
    otpo
    #get rid of weird quote marks
    converted_paths = [p.replace('"', '') for p in converted_paths]
    
    return converted_paths
                            